# Unit conversions

## Available conversions

Time-of-flight neutron-scattering unit conversions implement the following conversions.
For background we refer to PaN-learning on [Introduction to neutron scattering](https://pan-learning.org/wiki/index.php/Introduction_to_neutron_scattering) and [Basics of neutron scattering](https://pan-learning.org/wiki/index.php/Basics_of_neutron_scattering).
In scippneutron, unit conversions are performed by [scippneutron.convert](../generated/scippneutron.convert.rst#scippneutron.convert).

In summary, in the table below for the supported unit conversions:

- $h$ is the Planck constant, $m_{\mathrm{n}}$ is the neutron mass.
- $L_1$ is the primary flight path length, i.e., the distance between source and sample.
  The source position defines the point where $t=0$ (or vice versa).
- $L_2$ is the secondary flight path length, i.e., the distance(s) between sample and detector(s).
- $L_{\mathrm{total}} = L_1 + L_2$
- $d$ is the interplanar lattice spacing
- $\theta$ is the scattering angle, as defined in Bragg's law, not to be confused with $\theta_{\mathrm{spherical}}$ in sperical coordinates.
  In many instruments the beam is roughly aligned with the $z$-axis, so $2\theta = \theta_{\mathrm{spherical}}$.
  The factor of $2$ is a recurrent source of bugs.
  
In the special case of inelastic neutron scattering we have additionally:

- $E_i$ is the known incident energy. This case is called *direct inelastic scattering*.
  We define $t_0 = \sqrt{L_1^2 \cdot m_{\mathrm{n}} / E_i}$
- $E_f$ is the known final energy. This case is called *indirect inelastic scattering*.
  We define $t_0 = \sqrt{L_2^2 \cdot m_{\mathrm{n}} /E_F}$
  In this case $E_f$ is actually determined by analyzer crystals in the secondary flight path.
  It is assumed that the detector positions are set to the effective (neutronic) rather than physical positions, since the computation of $L_2$ assumes a straight line connection between sample and detectors.

|Origin unit |Target unit |Formula  used for coord conversion |
|---|---|:---|
|`tof`|`dspacing`|$d = \frac{\lambda}{2\sin(\theta)} = \frac{h \cdot t}{L_{\mathrm{total}} \cdot m_{\mathrm{n}} \cdot 2 \sin(\theta)}$|
| `tof`|`wavelength`|$\lambda = \frac{h \cdot t}{m_{\mathrm{n}} \cdot L_{\mathrm{total}}}$|
|`wavelength`|`Q`|$Q = \frac{4 \pi \sin(\theta)}{\lambda}$|
|`Q`|`wavelength`|$\lambda = \frac{4 \pi \sin(\theta)}{Q}$|
| `tof`|`energy`|$E = \frac{m_{\mathrm{n}}L^2_{\mathrm{total}}}{2t^2}$|
| `tof`|`energy_transfer` (direct)|$E = E_i - \frac{m_{\mathrm{n}}L^2_{\mathrm{2}}}{2\left(t-t_0\right)^{2}}$|
| `tof`|`energy_transfer` (indirect)|$E = \frac{m_{\mathrm{n}}L^2_{\mathrm{1}}}{2\left(t-t_0\right)^{2}} - E_f$|

The parameters given in the table above are retrieved from the coordinates or attributes of the input data.
`convert` searches through a graph to determine which coords (or attrs) to use.
The exact graph depends on the chosen `origin` and `target` coordinates.
You can get the graph for a concrete conversion using [scippneutron.conversion_graph](../generated/scippneutron.conversion_graph.rst#scippneutron.conversion_graph).

## Elastic scattering conversions
The graph below is a representative example for conversions from `tof` to elastic-scattering coordinates.

In [ ]:
import scipp as sc
import scippneutron as scn

In [ ]:
sc.show_graph(scn.conversion_graph(origin='tof',
                                   target='energy',
                                   scatter=True,
                                   energy_mode='elastic'),
              simplified=True)

Incoming arrows show inputs of a quantity.
For example, `wavelength` has `Ltotal` and `tof` as inputs.
Those must either be stored in the input data or there needs to be a way of computing them from their respective inputs.

To illustrate, consider a conversion from `tof` to `energy`. `convert` will proceed in the following way
- If `energy` is present as a coordinate or attribute, no action is required.
- Otherwise, compute `energy` from `tof` and `Ltotal`.
  - If `tof` is present, use that, else abort as `tof` cannot be computed from other parameters.
  - If `Ltotal` is present, use that, else compute from `L1` and `L2`.
    - Proceed with `L1` and `L2` in the same manner, tying to read them from the coordinates or attributes or compute them from their inputs.

In the following, we start with coorindates for `sample_position`, `position`, `incident_beam`, and `tof` and use those to compute `energy`.

In [ ]:
da = sc.DataArray(sc.arange('x', 1, 7,
                            unit='counts').fold('x', {
                                'spectrum': 2,
                                'tof': 3
                            }),
                  coords={
                      'tof':
                          sc.array(dims=['tof'],
                                   values=[5300.0, 6000.0, 6100.0, 7300.0],
                                   unit='us'),
                      'sample_position':
                          sc.vector(value=[0.0, 0.0, 0.0], unit='m'),
                      'position':
                          sc.vectors(dims=['spectrum'],
                                     values=[[1.0, 0.0, 0.0], [0.1, 0.0, 1.0]],
                                     unit='m'),
                      'incident_beam':
                          sc.vector(value=[0.0, 0.0, 10.0], unit='m')
                  })
da

In [ ]:
scn.convert(da, origin='tof', target='energy', scatter=True)

Note how `convert` produced the desired `energy` coordinate and renamed the old `tof` dimension to `energy`.
It also saved all intermediate results along with the inputs as attributes.

## Inelastic scattering conversions

As explained above, there are two cases of inelastic scattering, direct and indirect.
`convert` determines which one to use based on the available coordinates.
If the input has a coordinate 
- `incident_energy` &rarr; use direct inelastic scattering
- `final_energy` &rarr; use indirect inelastic scattering
- both &rarr; error because the situation is ambiguous
- neither &rarr; error because inputs are missing

The graph below is for direct inelastic scattering.
The graph for the indirect case is analogous.

In [ ]:
sc.show_graph(scn.conversion_graph(origin='tof',
                                   target='energy_transfer',
                                   scatter=True,
                                   energy_mode='direct_inelastic'),
              simplified=True)

## Non-scattering conversion

When converting coordinates of measurements without scattering, for example of beam monitors on imaging beamlines, a different graph needs to be used as there is no scattered beam.

In [ ]:
sc.show_graph(scn.conversion_graph(origin='tof',
                                   target='energy',
                                   scatter=False,
                                   energy_mode='elastic'),
              simplified=True)

`scippneutron.convert` uses [scipp.transform_coords](https://scipp.github.io/user-guide/coordinate-transformations.html) under the hood.
See the linked page for more details on conversion graphs and how to construct your own.